In [ ]:
import fitz
import json
import time
from requests import get, post
from PIL import Image
import io

In [ ]:
endpointnew="https://*************.cognitiveservices.azure.com/"

post_url1 = endpointnew + "/formrecognizer/v2.1/prebuilt/receipt/analyze"

apim_key1="*********"

In [ ]:
source="  .pdf"

In [ ]:
headers = {
    # Request headers
    'Content-Type':'application/pdf' ,#'application/pdf','image/jpeg'
    'Ocp-Apim-Subscription-Key': apim_key1,
}

params = {
    "includeTextDetails": True,
    "locale":"en-IN"
}

with open(source, "rb") as f:
    data_bytes = f.read()

try:
    resp = post(url = post_url1, data = data_bytes, headers = headers, params = params)
    if resp.status_code != 202:
        print("POST analyze failed:\n%s" % resp.text)
        quit()
    print("POST analyze succeeded:\n%s" % resp.headers)
    get_url = resp.headers["operation-location"]
except Exception as e:
    print("POST analyze failed:\n%s" % str(e))
    quit()

n_tries = 10
n_try = 0
wait_sec = 6
while n_try < n_tries:
    try:
        resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key1})
        resp_json = json.loads(resp.text)
        if resp.status_code != 200:
            print("GET Layout results failed:\n%s" % resp_json)
            quit()
        status = resp_json["status"]
        file = open(source.split('.')[0]+"receipt_newmodel.txt", "w")
        file.write(json.dumps(resp.json())) 
        file.close()
        if status == "succeeded":
            print("Layout Analysis succeeded:\n%s" % resp_json)
            break
        if status == "failed":
            print("Analysis failed:\n%s" % resp_json)
            quit()
        # Analysis still running. Wait and retry.
        time.sleep(wait_sec)
        n_try += 1     
    except Exception as e:
        msg = "GET analyze results failed:\n%s" % str(e)
        print(msg)
        quit()

In [ ]:
resultdict={}
itemdict={}
for pageresult in resp_json["analyzeResult"]["documentResults"]:
    for ii,vv in pageresult['fields'].items():
        print(ii,vv)
        if 'valueString' in vv.keys():
            resultdict[ii]=[]
        #print(vv)
            resultdict[ii].append(vv['valueString'])
            print(resultdict)
        elif 'valueArray' in vv.keys():
            for iii in vv['valueArray']:
                resultdict[ii]={}
                for kk in iii['valueObject'].keys():
                    #print(resultdict[ii].keys())
                    if kk not in itemdict.keys():
                        itemdict[kk]=[]
                        print(kk)
                        if 'type' in iii['valueObject'][kk].keys() == 'string':
                         
                            itemdict[kk].append(iii['valueObject'][kk]['valueString'])
                        else:
                            try:
                                itemdict[kk].append(iii['valueObject'][kk]['text'])   
                            except:
                                    
                                itemdict[kk].append(iii['valueObject'][kk]['valueNumber']) 
                    else:
                        
                        
                        if 'type' in iii['valueObject'][kk].keys() == 'string':
                         
                            itemdict[kk].append(iii['valueObject'][kk]['valueString'])
                        else:
                            try:
                                itemdict[kk].append(iii['valueObject'][kk]['text'])   
                            except:
                                    
                                itemdict[kk].append(iii['valueObject'][kk]['valueNumber']) 
        elif 'valueNumber' in vv.keys():
            if ii not in resultdict.keys():
                resultdict[ii]=[]
                resultdict[ii].append(vv['valueNumber'])
            else:
                resultdict[ii].append(vv['valueNumber'])
        elif 'phoneNumber' in vv.keys():
            
            if ii not in resultdict.keys():
                resultdict[ii]=[]
                resultdict[ii].append(vv['valuePhoneNumber'])
            else:
                resultdict[ii].append(vv['valuePhoneNumber'])
        elif 'valuePhoneNumber' in vv.keys():
            if ii not in resultdict.keys():
                resultdict[ii]=[]
                resultdict[ii].append(vv['valuePhoneNumber'])
            else:
                resultdict[ii].append(vv['valuePhoneNumber'])
        elif 'valueDate' in vv.keys():
            if ii not in resultdict.keys():
                resultdict[ii]=[]
                resultdict[ii].append(vv['valueDate'])
            else:
                resultdict[ii].append(vv['valueDate'])       
            
        else:
            continue
                    
        
if 'Items' in resultdict.keys():
    resultdict['Items']=itemdict
    
    

In [ ]:
resultdict